In [1]:
# call relevant packages
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp

# to use GPU 
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.1.0
Apache Spark version:  3.1.2


## Creating a composite CoNLL file 

### Create training set

In [2]:
# create the training filJe with all the news corpus
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/onto.bn.ner") as fp:
    text = fp.readlines()

In [3]:
text = "".join(text[1:]).split("\n\n") 

In [4]:
text[:10]

['This\tDT\t(TOP(S(NP*)\tO\nis\tVBZ\t(VP*\tO\nThe\tDT\t(NP(NP*\tB-ORG\nWorld\tNNP\t*)\tI-ORG\n,\t,\t*\tO\na\tDT\t(NP(NP*\tO\nco-production\tNN\t*)\tO\nof\tIN\t(PP*\tO\nthe\tDT\t(NP(NP*\tB-ORG\nBBC\tNNP\t*\tI-ORG\nWorld\tNNP\t*\tI-ORG\nService\tNNP\t*)\tI-ORG\n,\t,\t*\tO\nPRI\tNNP\t(NP*)\tB-ORG\n,\t,\t*\tO\nand\tCC\t*\tO\nWGBH\tNNP\t(NP(NP*)\tB-ORG\nin\tIN\t(PP*\tO\nBoston\tNNP\t(NP*))))))))\tB-GPE\n.\t.\t*))\tO',
 'I\tPRP\t(TOP(S(NP*)\tO\nam\tVBP\t(VP*\tO\nLisa\tNNP\t(NP*\tB-PERSON\nMullins\tNNP\t*))\tI-PERSON\n.\t.\t*))\tO',
 "A\tDT\t(TOP(S(NP*\tO\nplan\tNN\t*)\tO\nwas\tVBD\t(VP*\tO\nannounced\tVBN\t(VP*\tO\ntoday\tNN\t(NP*)\tB-DATE\nto\tTO\t(S(VP*\tO\nraise\tVB\t(VP*\tO\nthe\tDT\t(NP(NP*\tO\nRussian\tJJ\t*\tB-NORP\nnuclear\tJJ\t*\tO\nsubmarine\tNN\t*)\tO\n`\t''\t(NP*\tO\nKursk\tNNP\t*\tB-PRODUCT\n'\t''\t*))\tO\n,\t,\t*\tO\nnext\tJJ\t(NP(NP*\tB-DATE\nsummer\tNN\t*)\tI-DATE\n,\t,\t*\tO\nalmost\tRB\t(SBAR(NP(QP*\tB-DATE\na\tDT\t*)\tI-DATE\nyear\tNN\t*)\tI-DATE\nafter\tIN\t*\tO\nit\tPRP\

In [5]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [6]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [7]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/onto.mz.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [8]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [9]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/onto.nw.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [10]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [11]:
# adding tc files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/onto.tc.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [12]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [13]:
# adding wb files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/onto.wb.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [14]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [15]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/train/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### Create dev set

In [16]:
# create the training filJe with all the news corpus
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/development/onto.bc.ner") as fp:
    text = fp.readlines()

In [17]:
text = "".join(text[1:]).split("\n\n") 

In [18]:
text[:10]

['In\tIN\t(TOP(S(PP*\tO\nthe\tDT\t(NP(NP*\tB-DATE\nsummer\tNN\t*)\tI-DATE\nof\tIN\t(PP*\tI-DATE\n2005\tCD\t(NP*))))\tI-DATE\n,\t,\t*\tO\na\tDT\t(NP(NP*\tO\npicture\tNN\t*)\tO\nthat\tWDT\t(SBAR(WHNP*)\tO\npeople\tNNS\t(S(NP*)\tO\nhave\tVBP\t(VP*\tO\nlong\tRB\t(ADVP*)\tO\nbeen\tVBN\t(VP*\tO\nlooking\tVBG\t(VP*\tO\nforward\tRB\t(ADVP*\tO\nto\tIN\t(PP*))))))))\tO\nstarted\tVBD\t(VP*\tO\nemerging\tVBG\t(S(VP*\tO\nwith\tIN\t(PP*\tO\nfrequency\tNN\t(NP*))\tO\nin\tIN\t(PP*\tO\nvarious\tJJ\t(NP*\tO\nmajor\tJJ\t*\tO\nHong\tNNP\t(NML*\tB-GPE\nKong\tNNP\t*)\tI-GPE\nmedia\tNNS\t*)))))\tO\n.\t.\t*))\tO',
 'With\tIN\t(TOP(S(PP*\tO\ntheir\tPRP$\t(NP*\tO\nunique\tJJ\t*\tO\ncharm\tNN\t*))\tO\n,\t,\t*\tO\nthese\tDT\t(NP*\tO\nwell\tRB\t(ADJP*\tO\n-\tHYPH\t*\tO\nknown\tVBN\t*)\tO\ncartoon\tNN\t*\tO\nimages\tNNS\t*)\tO\nonce\tRB\t(ADVP*\tO\nagain\tRB\t*)\tO\ncaused\tVBD\t(VP*\tO\nHong\tNNP\t(S(NP*\tB-GPE\nKong\tNNP\t*)\tI-GPE\nto\tTO\t(VP*\tO\nbe\tVB\t(VP*\tO\na\tDT\t(NP(NP*\tO\nfocus\tNN\t*)\tO\nof\tIN\t(P

In [19]:
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [20]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [21]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/development/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### Create test set

In [22]:
# create the training filJe with all the news corpus
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.bn.ner") as fp:
    text = fp.readlines()

In [23]:
text = "".join(text[1:]).split("\n\n") 

In [24]:
text[:10]

['Iraqi\tJJ\t(TOP(S(NP(NML*\tB-NORP\nleader\tNN\t*)\tO\nSaddam\tNNP\t*\tB-PERSON\nHussein\tNNP\t*)\tI-PERSON\nhas\tVBZ\t(VP*\tO\ngiven\tVBN\t(VP*\tO\na\tDT\t(NP*\tO\ndefiant\tJJ\t*\tO\nspeech\tNN\t*)\tO\nto\tTO\t(S(VP*\tO\nmark\tVB\t(VP*\tO\nthe\tDT\t(NP(NP*\tO\ntenth\tJJ\t*\tB-ORDINAL\nanniversary\tNN\t*)\tO\nof\tIN\t(PP*\tO\nthe\tDT\t(NP*\tB-EVENT\nGulf\tNNP\t*\tI-EVENT\nWar\tNNP\t*))))))))\tI-EVENT\n.\t.\t*))\tO',
 'He\tPRP\t(TOP(S(NP*)\tO\nsays\tVBZ\t(VP*\tO\nIraq\tNNP\t(SBAR(S(NP*)\tB-GPE\nhas\tVBZ\t(VP*\tO\ntriumphed\tVBN\t(VP*\tO\nover\tIN\t(PP*\tO\nthe\tDT\t(NP(NP*\tO\nevil\tNN\t*)\tO\nof\tIN\t(PP*\tO\nthe\tDT\t(NP*\tO\nWest\tNNP\t*)))))))))\tB-LOC\n.\t.\t*))\tO',
 'Barbara\tNNP\t(TOP(S(NP*\tB-PERSON\nPlett\tNNP\t*)\tI-PERSON\nreports\tVBZ\t(VP*\tO\nfrom\tIN\t(PP*\tO\nBaghdad\tNNP\t(NP*)))\tB-GPE\n.\t.\t*))\tO',
 'Saddam\tNNP\t(TOP(S(NP*\tB-PERSON\nHussein\tNNP\t*)\tI-PERSON\naddressed\tVBD\t(VP*\tO\nthe\tDT\t(NP*\tO\nnation\tNN\t*)\tO\nin\tIN\t(PP*\tO\na\tDT\t(NP(NP*\tO\nspeec

In [25]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [26]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [27]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.mz.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [28]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [29]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.nw.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [30]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [31]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### import data in CONLL format

In [32]:
# importing the training set
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/train/sample.train')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|This is The World...|[{document, 0, 88...|[{document, 0, 88...|[{token, 0, 3, Th...|[{pos, 0, 3, DT, ...|[{named_entity, 0...|
| I am Lisa Mullins .|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 0, I,...|[{pos, 0, 0, PRP,...|[{named_entity, 0...|
|A plan was announ...|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 0, A,...|[{pos, 0, 0, DT, ...|[{named_entity, 0...|
|The Kursk Foundat...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|The BBC 's James ...|[{document, 0, 66...|[{document, 0, 66...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ..

In [33]:
dev_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/development/sample.train')
dev_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|In the summer of ...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 1, In...|[{pos, 0, 1, IN, ...|[{named_entity, 0...|
|With their unique...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 3, Wi...|[{pos, 0, 3, IN, ...|[{named_entity, 0...|
|The world 's fift...|[{document, 0, 65...|[{document, 0, 65...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|The most importan...|[{document, 0, 67...|[{document, 0, 67...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|Well , for severa...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 3, We...|[{pos, 0, 3, UH, ..

In [34]:
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train')
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Iraqi leader Sadd...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 4, Ir...|[{pos, 0, 4, JJ, ...|[{named_entity, 0...|
|He says Iraq has ...|[{document, 0, 53...|[{document, 0, 53...|[{token, 0, 1, He...|[{pos, 0, 1, PRP,...|[{named_entity, 0...|
|Barbara Plett rep...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 6, Ba...|[{pos, 0, 6, NNP,...|[{named_entity, 0...|
|Saddam Hussein ad...|[{document, 0, 98...|[{document, 0, 98...|[{token, 0, 5, Sa...|[{pos, 0, 5, NNP,...|[{named_entity, 0...|
|`` Iraq has trium...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 1, ``...|[{pos, 0, 1, ``, ..

### Training the model - 10 epochs

In [35]:
# use bert embeddings
bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(True)#.setMaxSentenceLength(512)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [36]:
# transforming the training data into embeddings and saving it as parquet files
readyTrainingData = bert.transform(training_data)

readyTrainingData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_train")

In [37]:
readyTrainingData = spark.read.parquet("/tmp/conll2003/bert_train")

In [38]:
# transforming the development data into embeddings and saving it as parquet files
readyDevData = bert.transform(dev_data)

readyDevData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_dev")

In [39]:
readyDevData = spark.read.parquet("/tmp/conll2003/bert_dev")

In [40]:
# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

In [41]:
readyTestData = spark.read.parquet("/tmp/conll2003/bert_test")

### Dev set - news dataset

In [42]:
# initialize NER tagger
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(10)\
.setBatchSize(4)\
.setEnableMemoryOptimizer(True)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("/tmp/conll2003/bert_dev")

In [43]:
# train the model
%time myNerModel = nerTagger.fit(readyTrainingData)

CPU times: user 870 ms, sys: 478 ms, total: 1.35 s
Wall time: 2h 45min 33s


### Testing on News Dataset

In [44]:
# infer from the trained model
%time results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()

CPU times: user 2.2 s, sys: 369 ms, total: 2.57 s
Wall time: 21.2 s


In [45]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [46]:
print(count)
print(indices)

2
[146, 334]


In [47]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [48]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [49]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9745594261157319


In [50]:
print(f1_score(labels,ners))

0.8826413454974243


In [51]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.85      0.87      0.86       616
        DATE       0.85      0.87      0.86      1252
       EVENT       0.71      0.54      0.62        37
         FAC       0.49      0.67      0.56        66
         GPE       0.96      0.93      0.95      1664
    LANGUAGE       0.83      0.50      0.62        10
         LAW       0.82      0.39      0.53        36
         LOC       0.74      0.79      0.77       144
       MONEY       0.88      0.89      0.88       279
        NORP       0.96      0.96      0.96       579
     ORDINAL       0.80      0.88      0.84       118
         ORG       0.90      0.87      0.88      1494
     PERCENT       0.92      0.92      0.92       293
      PERSON       0.91      0.97      0.94      1099
     PRODUCT       0.60      0.75      0.66        71
    QUANTITY       0.70      0.80      0.75        59
        TIME       0.60      0.61      0.60       106
 WORK_OF_ART       0.42    

### Testing on News Conversations

In [52]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.bc.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [53]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [54]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

In [55]:
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train')
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|-- basically , it...|[{document, 0, 78...|[{document, 0, 78...|[{token, 0, 1, --...|[{pos, 0, 1, :, {...|[{named_entity, 0...|
|To express its de...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 1, To...|[{pos, 0, 1, TO, ...|[{named_entity, 0...|
|It takes time to ...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 1, It...|[{pos, 0, 1, PRP,...|[{named_entity, 0...|
|Dear viewers , th...|[{document, 0, 52...|[{document, 0, 52...|[{token, 0, 3, De...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
|     This is Xu Li .|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 3, Th...|[{pos, 0, 3, DT, ..

In [56]:
# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

In [57]:
# infer from the trained model
%time results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()

CPU times: user 1.23 s, sys: 309 ms, total: 1.54 s
Wall time: 55.9 s


In [58]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [59]:
print(count)
print(indices)

0
[]


In [60]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [61]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [62]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.976606747106624


In [63]:
print(f1_score(labels,ners))

0.7860340196956133


In [64]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.69      0.74      0.71       182
        DATE       0.67      0.67      0.67       200
       EVENT       0.40      0.14      0.21        14
         FAC       0.89      0.88      0.88        48
         GPE       0.93      0.90      0.92       353
         LAW       1.00      0.00      0.00         3
         LOC       0.79      0.73      0.76        26
       MONEY       1.00      1.00      1.00         3
        NORP       0.89      0.91      0.90       138
     ORDINAL       0.88      0.72      0.79        50
         ORG       0.78      0.78      0.78       153
     PERCENT       0.93      0.93      0.93        14
      PERSON       0.87      0.87      0.87       382
     PRODUCT       0.00      1.00      0.00         0
    QUANTITY       0.23      0.23      0.23        40
        TIME       0.51      0.43      0.47        63
 WORK_OF_ART       0.20      0.11      0.14        28

   micro avg       0.80   

### Testing on WebLogs

In [65]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.wb.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [66]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [67]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

In [68]:
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train')
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The success of al...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|The Source of the...|[{document, 0, 23...|[{document, 0, 23...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|The al - Jazeera ...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|In this film the ...|[{document, 0, 73...|[{document, 0, 73...|[{token, 0, 1, In...|[{pos, 0, 1, IN, ...|[{named_entity, 0...|
|The Hebrew channe...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ..

In [69]:
# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

In [70]:
# infer from the trained model
%time results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()

CPU times: user 381 ms, sys: 44 ms, total: 425 ms
Wall time: 31.6 s


In [71]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [72]:
print(count)
print(indices)

0
[]


In [73]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [74]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [75]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9680126682501979


In [76]:
print(f1_score(labels,ners))

0.7890556045895851


In [77]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.86      0.80      0.83        85
        DATE       0.58      0.74      0.65        74
       EVENT       0.43      0.25      0.32        12
         FAC       0.59      0.56      0.57        18
         GPE       0.92      0.93      0.93       173
    LANGUAGE       1.00      0.50      0.67         4
         LAW       1.00      0.00      0.00         1
         LOC       0.36      0.44      0.40         9
       MONEY       0.78      0.72      0.75        25
        NORP       0.90      0.89      0.90       107
     ORDINAL       0.75      0.50      0.60        18
         ORG       0.76      0.75      0.76       117
     PERCENT       0.64      0.64      0.64        33
      PERSON       0.86      0.82      0.84       407
     PRODUCT       0.00      0.00      0.00         1
    QUANTITY       0.20      0.33      0.25         6
        TIME       0.65      0.85      0.74        20
 WORK_OF_ART       0.41    

### Testing on Telephonic Conversations

In [78]:
# adding other news files
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/onto.tc.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [79]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [80]:
with open("/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

In [81]:
test_data = CoNLL().readDataset(spark, '/Users/ramyabala/Research Projects/Evaluate NER/bio/test/sample.train')
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|But %um , guessed...|[{document, 0, 32...|[{document, 0, 32...|[{token, 0, 2, Bu...|[{pos, 0, 2, CC, ...|[{named_entity, 0...|
|              What ?|[{document, 0, 5,...|[{document, 0, 5,...|[{token, 0, 3, Wh...|[{pos, 0, 3, WP, ...|[{named_entity, 0...|
|         %um The %um|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 2, %u...|[{pos, 0, 2, UH, ...|[{named_entity, 0...|
|             Again ?|[{document, 0, 6,...|[{document, 0, 6,...|[{token, 0, 4, Ag...|[{pos, 0, 4, RB, ...|[{named_entity, 0...|
|%um 118.91_120.82...|[{document, 0, 32...|[{document, 0, 32...|[{token, 0, 2, %u...|[{pos, 0, 2, UH, ..

In [82]:
# transforming the test data into embeddings and saving it as parquet files
readyTestData = bert.transform(test_data)

readyTestData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_test")

In [83]:
# infer from the trained model
%time results = myNerModel.transform(readyTestData).select("sentence","token","label","ner").collect()

CPU times: user 319 ms, sys: 29.7 ms, total: 349 ms
Wall time: 29.4 s


In [84]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [85]:
print(count)
print(indices)

0
[]


In [86]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [87]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [88]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9793185131195336


In [89]:
print(f1_score(labels,ners))

0.7272727272727273


In [90]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.59      0.44      0.51        52
        DATE       0.75      0.74      0.75        74
       EVENT       0.00      1.00      0.00         0
         FAC       0.25      0.33      0.29         3
         GPE       0.87      0.90      0.88        50
    LANGUAGE       1.00      0.50      0.67         8
         LOC       0.00      1.00      0.00         0
       MONEY       1.00      0.71      0.83         7
        NORP       0.81      1.00      0.89        17
     ORDINAL       0.89      0.89      0.89         9
         ORG       0.62      0.19      0.29        27
     PERCENT       0.80      0.67      0.73         6
      PERSON       0.86      0.92      0.89       100
     PRODUCT       0.20      0.25      0.22         4
    QUANTITY       0.00      1.00      0.00         0
        TIME       0.50      0.17      0.26        23
 WORK_OF_ART       0.00      1.00      0.00         0

   micro avg       0.76   